# Menambah Library

Memasukan library yang dibutuhkan dalam projek

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# AUGMENTASI GAMBAR

Generator untuk membuat gambar menjadi lebih variatif dan menentukan besar data yang digunakan untuk validasi

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split = 0.2
)

# DATASET

Ubah data menjadi dataset training & validation

In [ ]:
dataset = 'C:/Users/Hanif/Downloads/Compressed/coba'

jumlah_batch = 32
ukuran_gambar = (85, 85)

train = datagen.flow_from_directory(
    dataset,
    target_size=ukuran_gambar,
    batch_size=jumlah_batch,
    class_mode='categorical',
    subset='training'
)

val = datagen.flow_from_directory(
    dataset,
    target_size=ukuran_gambar,
    batch_size=jumlah_batch,
    class_mode='categorical',
    subset='validation'
)

# MODEL CNN

Membuat model dengan algoritma cnn untuk klasifikasi gambar multi kelas

In [ ]:
jumlah_kelas = val.num_classes

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(ukuran_gambar[0], ukuran_gambar[1], 3)))
model.add(MaxPool2D(2, 2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(2, 2))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(2, 2))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(2, 2))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(jumlah_kelas, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# LATIH

Model yang sudah dibuat dilatih memakai dataset training Dan validation

In [ ]:
akurasi_minimal = 85e-2
jumlah_langkah = int(train.samples/train.batch_size)

class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('val_accuracy') >= akurasi_minimal:
            self.model.stop_training = True
            
latih = model.fit(
    train,
    steps_per_epoch=jumlah_langkah,
    epochs=500,
    validation_data=val,
    callbacks=[my_callbacks()]
)

Grafik riwayat pelatihan untuk melihat seberapa baik model dilatih

In [ ]:
akurasi = latih.history['val_accuracy']
loss = latih.history['val_loss']

plt.plot(akurasi, color='blue', label='Akurasi')
plt.plot(loss, color='red', label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Nilai')
plt.title('Riwayat Pelatihan Model')
plt.legend()
plt.show()

Cek akurasi akhir model

In [ ]:
hasil = model.evaluate(val)

accuracy = hasil[1]
print(f'\nAkurasi Model: {accuracy * 100:.2f} %')

Evaluasi model dengan data studi kasus nyata diluar dataset

In [ ]:
import os

daftar_kelas = list(val.class_indices.keys())

daftar_file = os.listdir('Examples')

for file in daftar_file:
    gambar = os.path.join('Examples/', file)

    img = tf.keras.utils.load_img(gambar, target_size=ukuran_gambar)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    jenis = daftar_kelas[tf.argmax(score)]

    display = tf.keras.utils.load_img(gambar, target_size=None)
    plt.imshow(display)
    plt.xlabel(f'Kucing {jenis}\n\nfile: {file}', fontsize=13)
    plt.show()